<a href="https://colab.research.google.com/github/AnandVP123/q1/blob/main/Assignment_3_vanilla_model_finaltamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf dakshina_dataset_v1.0.tar
#tamil

     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 53.3 MB/s 
     |████████████████████████████████| 144 kB 48.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=9b0de4486aa98e0ff0e815a5ca2bdd9f7a11fc06e2707f8a10df650342cffd53
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
--2022-05-08 17:20:33--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 142.251.107.128, 74.125.141.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===========

In [9]:
##########processingdakshin#source of the code is mentioned in the report
import pandas as pd
import os
import cv2
import pathlib
import numpy as np
import matplotlib.pyplot as plt
##classDP
class DP():
    def __init__(self, DATAPATH, source_lang = 'en', target_lang = "ta"):
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.trainpath = os.path.join(DATAPATH, target_lang, "lexicons", target_lang+".translit.sampled.train.tsv")
        self.valpath = os.path.join(DATAPATH, target_lang, "lexicons", target_lang+".translit.sampled.dev.tsv")
        self.testpath = os.path.join(DATAPATH, target_lang, "lexicons", target_lang+".translit.sampled.test.tsv")
        self.train = pd.read_csv(self.trainpath,sep="\t",names=["tgt", "src", "count"],)
        self.val = pd.read_csv(self.valpath,sep="\t",names=["tgt", "src", "count"],)
        self.test = pd.read_csv(self.testpath,sep="\t",names=["tgt", "src", "count"],)
        ##Traijning of Data & creating val/test data 
        self.traindata = self.preprocess(self.train["src"].to_list(), self.train["tgt"].to_list())
        (self.trencoderinput,self.train_decoder_input,self.train_decoder_target,self.source_vocab,self.target_vocab,) = self.traindata
        self.so_chartoint, self.source_int2char = self.source_vocab
        self.tar_chartoint, self.target_int2char = self.target_vocab
        self.valdata = self.encode(self.val["src"].to_list(),self.val["tgt"].to_list(),list(self.so_chartoint.keys()),list(self.tar_chartoint.keys()),so_chartoint=self.so_chartoint,tar_chartoint=self.tar_chartoint,)
        self.valencoderinput, self.val_decoder_input, self.val_decoder_target = self.valdata
        self.so_chartoint, self.source_int2char = self.source_vocab
        self.tar_chartoint, self.target_int2char = self.target_vocab
        self.testdata = self.encode(self.test["src"].to_list(),self.test["tgt"].to_list(),list(self.so_chartoint.keys()),list(self.tar_chartoint.keys()),so_chartoint=self.so_chartoint,tar_chartoint=self.tar_chartoint,)
        self.testencoderinput, self.test_decoder_input, self.test_decoder_target = self.testdata
        self.so_chartoint, self.source_int2char = self.source_vocab
        self.tar_chartoint, self.target_int2char = self.target_vocab
    def dictionary_lookup(self, vocab):
        char2int = dict([(char, i) for i, char in enumerate(vocab)])
        int2char = dict((i, char) for char, i in char2int.items())
        return char2int, int2char
    def encode(self, source, target, source_chars, target_chars, so_chartoint=None, tar_chartoint=None):
        num_encoder_tokens = len(source_chars)
        numdecodertok = len(target_chars)
        sourcelengthmax = max([len(txt) for txt in source])
        targetlength = max([len(txt) for txt in target])
        source_vocab, target_vocab = None, None
        if so_chartoint == None and tar_chartoint == None:
            print("Generating the dictionary lookups for character to integer mapping and back")
            so_chartoint, source_int2char = self.dictionary_lookup(source_chars)
            tar_chartoint, target_int2char = self.dictionary_lookup(target_chars)
            source_vocab = (so_chartoint, source_int2char)
            target_vocab = (tar_chartoint, target_int2char)
        encoderindata = np.zeros((len(source), sourcelengthmax, num_encoder_tokens), dtype="float32")
        decoderindata = np.zeros((len(source), targetlength, numdecodertok), dtype="float32")
        decodertardata = np.zeros((len(source), targetlength, numdecodertok), dtype="float32")
        for i, (input_text, target_text) in enumerate(zip(source, target)):
            for t, char in enumerate(input_text):
                encoderindata[i, t, so_chartoint[char]] = 1.0
            encoderindata[i, t + 1 :, so_chartoint[" "]] = 1.0
            for t, char in enumerate(target_text):
                decoderindata[i, t, tar_chartoint[char]] = 1.0
                if t > 0:
                    decodertardata[i, t - 1, tar_chartoint[char]] = 1.0
            decoderindata[i, t + 1 :, tar_chartoint[" "]] = 1.0
            decodertardata[i, t:, tar_chartoint[" "]] = 1.0
        if source_vocab != None and target_vocab != None:return (encoderindata,decoderindata,decodertardata,source_vocab,target_vocab,)
        else:return encoderindata, decoderindata, decodertardata
    def preprocess(self, source , target):
        source_chars = set()
        target_chars = set()
        source = [str(x) for x in source]
        target = [str(x) for x in target]
        source_words = []
        target_words = []
        for src, tgt in zip(source, target):
            tgt = "\t" + tgt + "\n"
            source_words.append(src)
            target_words.append(tgt)
            for char in src:
                if char not in source_chars:
                    source_chars.add(char)
            for char in tgt:
                if char not in target_chars:
                    target_chars.add(char)
        target_chars = sorted(list(target_chars))
        source_chars = sorted(list(source_chars))
        source_chars.append(" ")
        target_chars.append(" ")
        num_encoder_tokens = len(source_chars)
        print("Source Vocab length:", num_encoder_tokens)
        numdecodertok = len(target_chars)
        print("Target Vocab length:", numdecodertok)
        sourcelengthmax = max([len(txt) for txt in source_words])
        print("Max iplength:", sourcelengthmax)
        targetlength = max([len(txt) for txt in target_words])
        print("Max seqlength:", targetlength)
        print("Number of samples:", len(source))
        return self.encode(source_words, target_words, source_chars, target_chars)
#####processesing db for tamil
DATAPATH = "./dakshina_dataset_v1.0"
dataBase = DP(DATAPATH) 


Source Vocab length: 27
Target Vocab length: 49
Max iplength: 30
Max seqlength: 28
Number of samples: 68218
Generating the dictionary lookups for character to integer mapping and back


In [44]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model, Sequential,  Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Activation, LSTM, SimpleRNN, GRU, TimeDistributed
import numpy as np
import matplotlib.pyplot as plt
import os
import wandb
class SequencetoSequence():
    def __init__(self, modelConfigDict, srcChar2Int, tgtChar2Int, using_pretrained_model = False):
        #self.native_vocabulary = modelConfigDict["native_vocabulary"]
        self.numEn = modelConfigDict["numEn"]
        self.CT = modelConfigDict["CT"]
        self.Dimlat = modelConfigDict["Dimlat"]
        self.Dout = modelConfigDict["Dout"]
        self.numDe = modelConfigDict["numDe"]
        self.hidlayer = modelConfigDict["hidlayer"]
        self.tgtChar2Int = tgtChar2Int
        self.srcChar2Int = srcChar2Int
    def build_configurable_model(self):       
        if self.CT == "RNN":
            eninput = Input(shape=(None, len(self.srcChar2Int)))
            enout = eninput
            for i in range(1, self.numEn + 1):
                encoder = SimpleRNN(self.Dimlat,return_state=True,return_sequences=True,Dout=self.Dout,)
                enout, state = encoder(eninput)
            encoderstates = [state]
            decoderinput = Input(shape=(None, len(self.tgtChar2Int)))
            deout = decoderinput
            for i in range(1, self.numDe + 1):
                decoder = SimpleRNN(self.Dimlat,return_sequences=True,return_state=True,Dout=self.Dout,)
                deout, _ = decoder(decoderinput, initial_state=encoderstates)
            hidlayer = Dense(self.hidlayer, activation="relu")
            hiddenout = hidlayer(deout)
            decoddens = Dense(len(self.tgtChar2Int), activation="softmax")
            deout = decoddens(hiddenout)
            model = Model([eninput, decoderinput], deout)
            return model
        elif self.CT == "GRU":
            eninput = Input(shape=(None, len(self.srcChar2Int)))
            enout = eninput
            for i in range(1, self.numEn + 1):
                encoder = GRU(self.Dimlat,return_state=True,return_sequences=True,Dout=self.Dout,)
                enout, state = encoder(eninput)
            encoderstates = [state]
            decoderinput = Input(shape=(None, len(self.tgtChar2Int)))
            deout = decoderinput
            for i in range(1, self.numDe + 1):
                decoder = GRU(self.Dimlat,return_sequences=True,return_state=True,Dout=self.Dout,)
                deout, _ = decoder(decoderinput, initial_state=encoderstates)
            hidlayer = Dense(self.hidlayer, activation="relu")
            hiddenout = hidlayer(deout)
            decoddens = Dense(len(self.tgtChar2Int), activation="softmax")
            deout = decoddens(hiddenout)
            model = Model([eninput, decoderinput], deout)
            return model       
        elif self.CT == "LSTM":
            # encoder
            eninput = Input(shape=(None, len(self.srcChar2Int)))
            enout = eninput
            for i in range(1, self.numEn + 1):
                encoder = LSTM(self.Dimlat,return_state=True,return_sequences=True,Dout=self.Dout,)
                enout, state_h, state_c = encoder(enout)
            encoderstates = [state_h, state_c]
            decoderinput = Input(shape=(None, len(self.tgtChar2Int)))
            deout = decoderinput
            for i in range(1, self.numDe + 1):
                decoder = LSTM(self.Dimlat,return_state=True,return_sequences=True,Dout=self.Dout,)
                deout, _, _ = decoder(deout, initial_state=encoderstates)
            hidlayer = Dense(self.hidlayer, activation="relu")
            hiddenout = hidlayer(deout)
            decoddens = Dense(len(self.tgtChar2Int), activation="softmax")
            deout = decoddens(hiddenout)
            model = Model([eninput, decoderinput], deout)
            return model


In [45]:
from tensorflow.keras.layers import RNN, LSTM, GRU, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Input, Model
import numpy as np
import os
import pandas as pd
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
def train():
    config_defaults = {"celltype": "RNN","Dim": 256,"hidlayer": 128,"optimi": "rmsprop","numEncoders": 1,"numDecoders": 1,"dropout": 0.2,"epochs": 1,"batchsize": 64,}
    wandb.init(config=config_defaults,  project="cs6910_a3", entity="ee20d064oe21d019")
    config = wandb.config
    wandb.run.name = (str(config.CT)+ dataBase.source_lang+ str(config.numEn)+ "_"
        + dataBase.target_lang+ "_"+ str(config.numDe)+ "_"+ config.optimiser+ "_"
        + str(config.epochs)+ "_"+ str(config.Dout) + "_"+ str(config.batchsize)+ "_"+ str(config.Dimlat))
    model = modelInit.build_configurable_model()
    ##
    modelInit = SequencetoSequence(config,srcChar2Int=dataBase.source_char2int, tgtChar2Int=dataBase.target_char2int)
    model.summary()
    #model.compile(optimizer=config.optimiser,loss="categorical_crossentropy",metrics=["accuracy"],)    
    ##
    wandb.run.save()
    model.save(os.path.join("./TrainedModels", wandb.run.name))    
    wandb.finish()
    ##
    earlystopping = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto")
    model.fit([dataBase.train_encoder_input, dataBase.train_decoder_input],dataBase.train_decoder_target,batchsize=config.batchsize,
        epochs=config.epochs,validation_data=([dataBase.val_encoder_input, dataBase.val_decoder_input], dataBase.val_decoder_target),callbacks=[earlystopping, WandbCallback()],)
    return model


In [40]:
sweep_config = {"name": "Bayesian Sweep without attention",
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {"CT": {"values": ["RNN", "GRU", "LSTM"]},"Dimlat": {"values": [256]},"hidlayer": {"values": [128, 64]},"optimiser": {"values": ["rmsprop", "adam"]},"numEn": {"values": [1, 2, 3]},        
        "numDe": {"values": [1, 2, 3]},"Dout": {"values": [0.1, 0.2, 0.3]},"epochs": {"values": [5,10,15, 20]},"batchsize": {"values": [32, 64]},},}
sweep_id = wandb.sweep(sweep_config, project="cs6910_a3", entity="ee20d064oe21d019")
wandb.agent(sweep_id, train, count = 200)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: ptwhqqdn
Sweep URL: https://wandb.ai/ee20d064oe21d019/cs6910_a3/sweeps/ptwhqqdn


wandb: Agent Starting Run: 1mv0y64x with config:
wandb: 	CT: RNN
wandb: 	Dimlat: 256
wandb: 	Dout: 0.2
wandb: 	batch_size: 64
wandb: 	epochs: 20
wandb: 	hidlayer: 128
wandb: 	numDe: 1
wandb: 	numEn: 2
wandb: 	optimiser: rmsprop
wandb: Currently logged in as: avp (ee20d064oe21d019). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
#afterlinking gdrive
#!cp -rf ./TrainedModels /content/gdrive/MyDrive/CS6910/Assignment3/